In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm
import os 

import matplotlib.pylab as pl

In [2]:
import axisrules as axis

In [3]:
path_dataset = "../data/" 

list_datasets = []

for folder in os.listdir(path_dataset):
    for dataset in os.listdir(path_dataset+folder):
        candidates = path_dataset+folder+"/candidates.csv"
        # Test isdir 
        if os.path.isdir(path_dataset+folder+"/"+dataset):
            path_weights=  path_dataset+folder+"/"+dataset+"/weights.csv" 
            # Test if there is weight file:
            isweight = False
            if os.path.isfile(path_weights):
                isweight = True
            for file in os.listdir(path_dataset+folder+"/"+dataset+"/votes"):
                if file.endswith(".csv"):
                    list_datasets.append({"votes":path_dataset+folder+"/"+dataset+"/votes/"+file, 
                                            "weights":path_weights,
                                            "isweight":isweight,
                                            "candidates": candidates,
                                            "year": folder,
                                            "dataset": dataset,
                                            "file":file})



            

In [4]:
axis_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, 
              axis.MinimalSwaps, axis.ForbiddenTriples]
axis_rules_seq = [axis.SequentialVoterDeletion, axis.SequentialMinimalFlips, 
              axis.SequentialBallotCompletion, axis.SequentialMinimalSwaps, axis.SequentialForbiddenTriples]

n_candidates = 12

tab_res = []

if n_candidates > 6:
    df_previous = pd.read_csv("data/french_axis_yes_w_%i.csv"%(n_candidates-1))
for data in list_datasets:

    if data["year"] not in ["France 2017", "France 2022"]:
        continue


    # filter the one with dataset and year and file:
    if n_candidates > 6:
        df_previous_data =  df_previous[(df_previous["dataset"]==data["dataset"]) & (df_previous["year"]==data["year"]) & (df_previous["file"]==data["file"])]



    # print("")
    print(data["dataset"], data["year"])
    dataset = data["dataset"]
    year = data["year"]
    df = pd.read_csv(data["votes"], index_col=0)
    candidates = df.columns
    votes = df.values
    n_voters = votes.shape[0]
    # print("voters:",votes.shape[0],"\n")
    weights = None 
    df_cand = pd.read_csv(data["candidates"], index_col=0)

    if True:
        if data["isweight"]:
            df_weights = pd.read_csv(data["weights"], index_col=0)
            weights = []
            if data["file"] == "approval.csv":
                w_type = "approval"
            else:
                w_type = "approval_with_runoff"
            for i in range(len(df.index)):
                ind = df.index[i]
                if ind in df_weights.index:
                    weights.append(df_weights.loc[ind][w_type])
                else:
                    weights.append(0)
            weights = np.array(weights)
    

    file = data["file"]
    dict_cand = df_cand[["name", "score"]].to_dict()
    cands_id = [e[1] for e in sorted([(dict_cand["score"][x],x) for x in dict_cand["score"]])[::-1][:n_candidates]]
    cands_name = [dict_cand["name"][x] for x in cands_id]
    selected_id = [i for i in range(len(candidates)) if candidates[i] in cands_name]
    votes = votes[:, selected_id]
    # count number of votes with > 2 approvals 
    active_voters = np.sum(np.sum(votes, axis=1)>=2)
    candidates = [candidates[i] for i in selected_id]
    n_candidates_bis = len(candidates)
    if n_candidates != n_candidates_bis:
        print(n_candidates_bis, cands_name)
        if dataset != "IEP Paris":
            continue

    
    for j,rule in enumerate(axis_rules):
        name = rule.name
        print(name)
        rule_instance = rule(votes, weights)

        if n_candidates > 6:
            df_previous_rule = df_previous_data[df_previous_data["rule"]==name]

            # get first element
            previous_axis = df_previous_rule.iloc[0]["axis"]
            axis_num = axis.reverse_axis(previous_axis, cands_name[:-1])
            min_score = None
            min_axis = None
            for i in range(len(axis_num)+1):
                tested_axis = axis_num[:i]+[n_candidates-1]+axis_num[i:]
                score_axis = rule_instance.get_score(tested_axis)
                if min_score is None or score_axis < min_score:
                    min_score = score_axis
                    min_axis = tested_axis


            # axis.print_axis(min_axis, cands_name)

        # seq_result = axis_rules_seq[j](votes, weights)()
        
        if n_candidates > 6:
            res = rule_instance.bruteforce(proxy_axis=min_axis)
        else:
            res = rule_instance.bruteforce()

        # axis.print_axis(res[0][0], candidates)
        axis_score = res[0][1]
        # print("Score of this axis :", int(axis_score))
        for el in res:
            tab_res.append([dataset, year, file, name, axis.get_axis(el[0], candidates), axis_score,n_voters,active_voters, n_candidates_bis])


df_res = pd.DataFrame(tab_res, columns=["dataset", "year", "file", "rule", "axis", "score", "n_voters","active_voters","n_candidates"])
df_res.to_csv("data/french_axis_yes_w_%i.csv"%n_candidates,index=False)


Crolles France 2017
11 ['Emmanuel Macron', 'Marine Le Pen', 'François Fillon', 'Jean-Luc Mélenchon', 'Benoît Hamon', 'Nicolas Dupont-Aignan', 'Jean Lassalle', 'Philippe Poutou', 'François Asselineau', 'Nathalie Arthaud', 'Jacques Cheminade']
Crolles France 2017
11 ['Emmanuel Macron', 'Marine Le Pen', 'François Fillon', 'Jean-Luc Mélenchon', 'Benoît Hamon', 'Nicolas Dupont-Aignan', 'Jean Lassalle', 'Philippe Poutou', 'François Asselineau', 'Nathalie Arthaud', 'Jacques Cheminade']
Grenoble France 2017
11 ['Emmanuel Macron', 'Marine Le Pen', 'François Fillon', 'Jean-Luc Mélenchon', 'Benoît Hamon', 'Nicolas Dupont-Aignan', 'Jean Lassalle', 'Philippe Poutou', 'François Asselineau', 'Nathalie Arthaud', 'Jacques Cheminade']
HSC France 2017
11 ['Emmanuel Macron', 'Marine Le Pen', 'François Fillon', 'Jean-Luc Mélenchon', 'Benoît Hamon', 'Nicolas Dupont-Aignan', 'Jean Lassalle', 'Philippe Poutou', 'François Asselineau', 'Nathalie Arthaud', 'Jacques Cheminade']
Online France 2017
11 ['Emmanuel Ma

100%|██████████| 3628800/3628800 [03:46<00:00, 16036.30it/s]


Minimal Flips


100%|██████████| 3628800/3628800 [10:00<00:00, 6039.72it/s] 


Ballot Completion


100%|██████████| 3628800/3628800 [03:39<00:00, 16498.93it/s]


Minimal Swaps


100%|██████████| 3628800/3628800 [08:39<00:00, 6991.29it/s] 


Forbidden Triples


100%|██████████| 3628800/3628800 [26:53<00:00, 2249.30it/s] 


In [34]:
axis_rules = [axis.VoterDeletion, axis.BallotCompletion]

for rule in axis_rules:
    print(rule.name)
    rule_instance = rule(votes, weights)
    res = rule_instance.lp()
    axis.print_axis(res, candidates)
    axis_score = rule_instance.get_score(res)   
    print("Score of this axis :", int(axis_score))

Voter Deletion


In [5]:

dict_dataset = {"France 2017":{}, "France 2022":{}}
for n_candidates in range(6,13):
    df_data = pd.read_csv("data/french_axis_yes_w_%i.csv"%(n_candidates))
    # filter the one with dataset and year and file:
    df_data =  df_data[(df_data["dataset"]=="Online")]

    for v in df_data.values:
            year = v[1]
            rule = v[3]
            if rule not in dict_dataset[year]:
                dict_dataset[year][rule] = []
            dict_dataset[year][rule].append((v[4],v[5],n_candidates))



In [9]:
def filter_upper(s):
    return ''.join([c for c in s if c.isupper() or c=="<"])

for year in dict_dataset:
    print(year)
    for rule in dict_dataset[year]:
        print(rule)
        for s, _, n  in dict_dataset[year][rule]:
            if n == 12:
                print(n, filter_upper(s))

France 2017
Voter Deletion
Minimal Flips
Ballot Completion
Minimal Swaps
Forbidden Triples
France 2022
Voter Deletion
12 JL<MLP<EZ<NDA<VP<EM<AH<YJ<JLM<PP<NA<FR
Minimal Flips
12 NDA<EZ<MLP<JL<VP<EM<YJ<AH<FR<JLM<PP<NA
Ballot Completion
12 NDA<EZ<MLP<JL<VP<EM<AH<YJ<JLM<FR<PP<NA
Minimal Swaps
12 NDA<EZ<MLP<JL<VP<EM<YJ<AH<JLM<FR<PP<NA
Forbidden Triples
12 NDA<EZ<MLP<JL<VP<EM<YJ<AH<FR<JLM<PP<NA
